In [1]:
%pip install -qU langchain langchain-openai langchain-anthropic langsmith python-dotenv pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

from langsmith import Client

client = Client()


In [3]:
example_system_prompt = """You are a helpful assistant

Rules:
- You are only allowed to talk about coding
- <HERE>
- Try to be concise"""

example_instruction = "The agent should only respond in English. No other languages"

examples = ["Hola, como estas?", "Hi there!"]

In [23]:
from typing import List
import random
from langsmith.schemas import Run, Example
from langsmith import evaluate
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic.v1 import BaseModel, Field
from langchain.chat_models import init_chat_model

# Schemas


class Rule(BaseModel):
    """A single rule for the prompt"""
    reasoning: str = Field(
        ..., description="The thought process and direction for why we think this is a good solution to the instruction.")
    prompt: str = Field(..., description="A single prompt rule that we can try to solve for the instruction. \
This prompt rule will be interpolated into the system prompt over the <HERE> token.")


class Rules(BaseModel):
    """Set of prompt rules that should be tried"""
    rules: List[Rule] = Field(
        ..., description="A list of prompt rules that we can try to solve for the instruction.")


class CorrectnessEvaluationResult(BaseModel):
    """Result of an evaluation of correctness"""
    reasoning: str = Field(
        ..., description="The thought process behind why you think the answer is correct or incorrect.")
    correct: bool = Field(..., description="Correctness score")


class Promptnado:
    def __init__(self, system_prompt: str, instruction: str, examples: List[str], rule_token="<HERE>"):
        self.system_prompt = system_prompt
        self.instruction = instruction
        self.examples = examples
        self.rule_token = rule_token

        # Create random dataset name
        self.dataset_name = f"Promptnado_{random.randint(0, 1000000)}"

        self.attempts = 1
        self.solved = False
        self.current_rule = None
        self.current_prompt = None
        self.successful_prompt = None

    def _create_dataset(self):
        """Create a dataset with a unique name"""
        dataset = client.create_dataset(
            self.dataset_name, description=self.instruction)
        for example in self.examples:
            client.create_example(
                inputs={"input": example}, dataset_id=dataset.id)

        self.dataset = dataset
        print(
            f"Created dataset: {self.dataset_name} with {len(self.examples)} examples")
        return dataset

    def _generate_rules(self):
        """Use an LLM to generate a list of rules"""

        system_prompt = f"""You are an expert LLM Prompt Engineer. Your job is to try to solve for the provided <Instructions> \
by making adjustments to the <Original Prompt>. You should attempt to make 5 suggestions for prompts that might work. Each suggestion you \
make will be interpolated into the prompt where {self.rule_token} is, and then evaluated for correctness against a dataset of \
examples.

<Instructions>
{self.instruction}
</Instructions>

<Original Prompt>
{self.system_prompt}
</Original Prompt>
"""

        structured_llm = init_chat_model(
            model="gpt-4o", temperature=0.7).with_structured_output(Rules)

        rules: Rules = structured_llm.invoke(system_prompt)

        self.rules = rules.rules
        print(f"Generated {len(self.rules)} rules\n")
        print(self.rules)
        return self.rules

    def _build_prompt(self, rule: Rule):
        """Interpolate the rules into the system prompt"""
        interpolated_prompt = self.system_prompt.replace(
            self.rule_token, rule.prompt)

        print(f"Interpolated prompt:\n\n{interpolated_prompt}")
        return interpolated_prompt

    def _evaluate_correctness(self, run: Run, example: Example):
        """Eval function to use an LLM to validate that the instruction was followed"""
        system_prompt = f"""Your job is to validate whether the <Result> meets the criteria for <Instruction>. Try to be a harsh judge.

<Instruction>
{self.instruction}
</Instruction>

<Result>
{run.outputs["output"]}
</Result>
"""

        structured_llm = init_chat_model(
            model="gpt-4o", temperature=0).with_structured_output(CorrectnessEvaluationResult)

        result: CorrectnessEvaluationResult = structured_llm.invoke(
            system_prompt)

        return {"score": 1 if result.correct else 0, "key": "correctness", "comment": result.reasoning}

    def _predict(self, inputs: dict):
        """Run current prompt against example in the dataset"""

        messages = [
            SystemMessage(content=self.current_prompt),
            HumanMessage(content=inputs["input"]),
        ]

        # Invoke the model
        llm = init_chat_model(model="gpt-4o", temperature=0.7)
        response = llm.invoke(messages)

        return {"output": response.content}
    
    def _is_solved(self, eval_results):
        """Validate the results"""

        results = eval_results._results

        # If any of the result scores are not a 1, return false
        for result in results:
            score = result['evaluation_results']["results"][0].score
            if score != 1:
                return False
            
        return True

    def _test_rule(self, rule: Rule):
        """Evaluate a given rule"""
        print(f"\nTesting rule: {rule.prompt}")
        self.current_rule = rule

        self.current_prompt = self._build_prompt(self.current_rule)

        results = evaluate(
            self._predict,
            data=self.dataset_name,
            evaluators=[self._evaluate_correctness],
            experiment_prefix=f"Attempt-{self.attempts}",
        )

        self.attempts += 1

        return results

    def run(self):
        """Run the promptnado"""
        print(f"Running Promptnado with instruction: {self.instruction}")
        # Create the dataset
        self._create_dataset()

        while self.solved == False:
            # Get a list of rules
            self._generate_rules()

            # For each rule
            for rule in self.rules:
                results = self._test_rule(rule)

                # Add results validation here
                if self._is_solved(results):
                    self.results = results
                    self.solved = True
                    self.successful_prompt = self.current_prompt
                    break

        print("\n\nSolved!! Current prompt can be found at `self.successful_prompt`\n\n")

        print(f"Successful prompt:\n====================\n{self.current_prompt}\n=================")

In [24]:
pn = Promptnado(example_system_prompt, example_instruction, examples)
pn.run()

Running Promptnado with instruction: The agent should only respond in English. No other languages
Created dataset: Promptnado_854125 with 2 examples
Generated 5 rules

[Rule(reasoning='By explicitly stating that the agent should respond only in English, we can ensure adherence to the instruction.', prompt='You should respond only in English.'), Rule(reasoning='This rule reinforces the language constraint by prohibiting responses in any other language.', prompt='Do not respond in any language other than English.'), Rule(reasoning='By specifying the language of responses, we can make it clear that English is the only acceptable language for communication.', prompt='All responses must be in English.'), Rule(reasoning='This rule makes it explicit that the agent must not use any language other than English.', prompt='Avoid any language other than English in your responses.'), Rule(reasoning='This rule emphasizes that English is the only language permitted for responses, ensuring clarity.', 

2it [00:03,  1.55s/it]



Solved!! Current prompt can be found at `self.successful_prompt`


Successful prompt:
You are a helpful assistant

Rules:
- You are only allowed to talk about coding
- You should respond only in English.
- Try to be concise


In [20]:
pn.current_prompt

'You are a helpful assistant\n\nRules:\n- You are only allowed to talk about coding\n- You should only respond in English.\n- Try to be concise'